In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
import numpy as np
import glob
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
import pickle
import tensorflow as tf
import re
import json
from sklearn.preprocessing import LabelEncoder, StandardScaler

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score
from keras.models import Sequential
from keras.layers import Dense

In [ ]:
# importing module
from pymongo import MongoClient
  
# creation of MongoClient
client=MongoClient()
  
# Connect with the portnumber and host
client = MongoClient("mongodb://localhost:27017/")
  
# Access database
mydatabase = client['Price_Predictor']
  
# Access collection of the database
col=mydatabase.car_details

data = []
cursor = col.find()
for cars in cursor:
    data.append(cars)

df = pd.DataFrame(data)

Cleaning Data

In [ ]:
df.isnull().sum()

In [ ]:
df = df.drop(['Make','Model','Mfg Exterior Colour','Passengers','Interior Colour','Hwy Fuel Economy','City Fuel Economy','MTW Status','Stock Number','Status','Trim','Engine','Exterior Colour' ],axis='columns')
df.head()

In [ ]:
df= df.dropna()

In [ ]:
df['Fuel Type'].unique()

In [ ]:
Gas = ['Gas','gas','Gasoline', 'gasoline', 'lead','lex','ropa']
for i in Gas:
    df["Fuel Type"] = np.where(df["Fuel Type"].str.contains('^.*{}.*$'.format(i), flags=re.IGNORECASE,regex=True) ,'Gasoline',df["Fuel Type"])

In [ ]:
Gas1 = ['Other/Don’t Know']
for i in Gas1:
    df["Fuel Type"] = np.where(df["Fuel Type"].str.contains(i, flags=re.IGNORECASE,regex=True) ,'Other',df["Fuel Type"])

In [ ]:
df['Fuel Type'].unique()

In [ ]:
df['Transmission'].unique()

In [ ]:
Trns = ['Automatic','Manual']
for i in Trns:
    df["Transmission"] = np.where(df["Transmission"].str.contains('^.*{}.*$'.format(i), flags=re.IGNORECASE,regex=True) ,i,df["Transmission"])

In [ ]:
df['Transmission'].unique()

In [ ]:
df['Kilometres'] = df['Kilometres'].str.slice(0,-3)
df['Kilometres'] = df['Kilometres'].str.replace(',','')

In [ ]:
df[['km', 'location','extra']] = df4["Car Location"].apply(lambda x: pd.Series(str(x).split("|")))

In [ ]:
df[['year', 'brande','name','extra']] = df4["car title"].apply(lambda x: pd.Series(str(x).split(" ",3)))

In [ ]:
df.drop(['Car Location','car title','extra','km','Doors','location'],axis=1,inplace=True)
df.head()

In [ ]:
df['Body Type'].unique()

In [ ]:
cab = ['cab','crew'] 
for i in cab:
    df["Body Type"] = np.where(df["Body Type"].str.contains('^.*{}.*$'.format(i), flags=re.IGNORECASE,regex=True) ,'Truck',df["Body Type"])

In [ ]:
cab = ['wagon'] 
for i in cab:
    df["Body Type"] = np.where(df["Body Type"].str.contains('^.*{}.*$'.format(i), flags=re.IGNORECASE,regex=True) ,'Wagon',df["Body Type"])

In [ ]:
cab = ['van','Cutaway'] 
for i in cab:
    df["Body Type"] = np.where(df["Body Type"].str.contains('^.*{}.*$'.format(i), flags=re.IGNORECASE,regex=True) ,'Van',df["Body Type"])

In [ ]:
cab = ['Compact'] 
for i in cab:
    df["Body Type"] = np.where(df["Body Type"].str.contains('^.*{}.*$'.format(i), flags=re.IGNORECASE,regex=True) ,'Sedan',df["Body Type"])

In [ ]:
cab = ['Roadster'] 
for i in cab:
    df["Body Type"] = np.where(df["Body Type"].str.contains('^.*{}.*$'.format(i), flags=re.IGNORECASE,regex=True) ,'Convertible',df["Body Type"])

In [ ]:
df['Body Type'].unique()

Converting Catogorical Values to numerical

In [ ]:
#create dictionaries
Body_Type = df['Body Type'].unique() 
Transmission = df['Transmission'].unique()  
Drivetrain = df['Drivetrain'].unique() 
Fuel_Type = df['Fuel Type'].unique() 
brande = df['brande'].unique() 
name = df['name'].unique()


def Convert(lst):
    res_dct = {lst[i]: i+1 for i in range(0, len(lst), 1)}
    return res_dct

Body_Type_dict = Convert(Body_Type)
Transmission_dict = Convert(Transmission)
Drivetrain_dict = Convert(Drivetrain)
Fuel_Type_dict = Convert(Fuel_Type)
brande_dict = Convert(brande)
name_dict = Convert(name)

cato_val  = {"body_type":Body_Type_dict,"transmission":Transmission_dict,"drivetrain":Drivetrain_dict,"fuel_type":Fuel_Type_dict,"brand":brande_dict,"name":name_dict}

with open("cato_val.json", "w") as outfile:
    outfile.write(json.dumps(cato_val, indent=4))
print(cato_val)

In [ ]:
# ['Body Type','Transmission','Drivetrain','Fuel Type','brande','name']
for key, value in Body_Type_dict.items():
    df['Body Type'] = np.where(df["Body Type"] == key,value,df['Body Type'])

for key, value in Transmission_dict.items():
    df['Transmission'] = np.where(df["Transmission"] == key,value,df['Transmission'])

for key, value in Drivetrain_dict.items():
    df['Drivetrain'] = np.where(df["Drivetrain"] == key,value,df['Drivetrain'])

for key, value in Fuel_Type_dict.items():
    df['Fuel Type'] = np.where(df["Fuel Type"] == key,value,df['Fuel Type'])

for key, value in brande_dict.items():
    df['brande'] = np.where(df["brande"] == key,value,df['brande'])

for key, value in name_dict.items():
    df['name'] = np.where(df["name"] == key,value,df['name'])

In [ ]:
# importing module
from pymongo import MongoClient
  
# creation of MongoClient
client=MongoClient()
  
# Connect with the portnumber and host
client = MongoClient("mongodb://localhost:27017/")
  
# Access database
mydatabase = client['Price_Predictor']
  
# Access collection of the database
col=mydatabase.car_converted_catogorical_data
  
for i,row in df.iterrows():
    record = row.to_json(orient = 'records')
    # inserting the data in the database
    mydatabase.car_details.insert_one(record)
